# Lock n load Table Scraper for Coursera

In [382]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [383]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [384]:
#importing libs and intalling if necessary
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd

In [385]:
#making a request to Wiki
html = urllib.request.urlopen("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
bs = html.read()

html_code = bs.decode("utf8")
html.close()

In [386]:
#extracting the table
soup = BeautifulSoup(html_code)

In [387]:
#dividing the table into 3 lists
table = soup.find('table',attrs = {'class':'wikitable sortable'})
tab = table.find_all('a')
td = table.find_all('td')
col1 = []
col2 = []
col3 = []
i = 0
for elem in td:
    if i == 0:
        col1.append(elem.text.split(',', 1))
        i+=1
    elif i == 1:
        col2.append(elem.text.split(',', 1))
        i+=1
    elif i == 2:
        col3.append(elem.text.split(',', 1))
        i = 0
final = []
for link in tab:
    final.append(link.get('title'))

In [388]:
e = []
s = {}
i = 0
for m in col1, col2, col3:
    e.append({str(i) : [''.join(i).replace('\n', '').strip() for i in m]})
    i+=1
#making a dictionary form the table
s.update(e[0])
s.update(e[1])
s.update(e[2])

In [389]:
#loading into Pandas DataFrame
df = pd.DataFrame(s)
df2 = df.copy()

In [390]:
#spotting duplicates
un = df['0'].duplicated(keep=False)       

In [391]:
#grouping values from duplicates
i = 1
m = {}
indexes = []
cities = []
temp=""
while i in range(len(un)):
    if un[i] == True:
        indexes.append(i)
        temp += df['2'][i] + ', '
        cities.append(str(df['1'][i]))
        i+=1
        while df['0'][i] == df['0'][i - 1]:
            temp += df['2'][i]
            indexes.append(i)
            i+=1
            if df['0'][i] == df['0'][i - 1]:
                temp += ', '
        m[df['0'][i - 1].replace('\n', '')] = temp
        temp=""
    if un[i] == False:    
        i+=1

In [392]:
#concating
lkeys = list(m.keys())
df3 = df3.transpose()
tab1 = {'0' : lkeys}
tab2 = {'2' : list(m.values())}
tab3 = {'1' : cities}
tab1.update(tab2)
tab1.update(tab3)
df3 = pd.DataFrame(tab1)

In [393]:
#dropping appropriates inside the copy of the dataframe
df2.drop(indexes, inplace=True)

In [394]:
#Assigning values which weren't assigned to Neighborhood from Borough
dfinal = 0
dfinal = pd.concat([df3,df2], sort='true', join='inner')
i = 0
for index, row in dfinal.iterrows():
    if row['1'] != 'Not assigned' and row['2'] == 'Not assigned':
        row['2'] = row['1']
    i+=1

In [395]:
#Naming the columns
dfinal.columns = ['PostalCode', 'Borough', 'Neighborhood']

dfinal = dfinal.reset_index()
del dfinal['index']

## **Here we are:**
-------------------

In [396]:
dfinal.head(12)

,PostalCode,Borough,Neighborhood
0,M5A,Downtown Toronto,"Harbourfront, Regent Park"
1,M6A,North York,"Lawrence Heights, Lawrence Manor"
2,M1B,Scarborough,"Rouge, Malvern"
3,M4B,East York,"Woodbine Gardens, Parkview Hill"
4,M5B,Downtown Toronto,"Ryerson, Garden District"
5,M9B,Etobicoke,"Cloverdale, Islington, Martin Grove, Princess ..."
6,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
7,M3C,North York,"Flemingdon Park, Don Mills South"
8,M9C,Etobicoke,"Bloordale Gardens, Eringate, Markland Wood, Ol..."
9,M1E,Scarborough,"Guildwood, Morningside, West Hill"


In [397]:
dfinal.shape

(180, 3)